<a href="https://colab.research.google.com/github/LuizFelipe-FF/coords-tab-geoquimica/blob/main/Espacializa%C3%A7%C3%A3o_Tabela_Geoqu%C3%ADmica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

___
*Autor: Luiz Felipe Franco Ferreira - Graduando em Geologia*

*Atividade: Seleção de dados e criação de um SHP*

*Data: 18/06/2025*

*Resumo: Código desenvolvido para garantir ID's únicos para cada amostra, padronizar os diversos tipos de coordenadas,criação de um Geodataframe agrupando as amostras que possuam Lat e Long, e gerar um SHP.*
___

###Etapas:
1. Padronização dos ID's
2. Padronização das Coords
3. Criação do GeoDataFrame

In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from pyproj import Proj
import re

###1. Padronização dos ID's
Impede de aparecer o mesmo ID para amostras distintas, criando assim a coluna sample name unique adicionando sufixos aos ID's.

In [2]:
# 1. Carregar CSV
df = pd.read_csv("pmp_recalculated.csv", sep=';')

# 2. Nomes únicos para amostras
def make_unique_names(series):
    counts = {}
    unique_names = []
    for name in series:
        if pd.isna(name):
            unique_names.append(name)
            continue
        count = counts.get(name, 0)
        suffix = '' if count == 0 else f'_{chr(96 + count + 1)}'
        unique_names.append(f"{name}{suffix}")
        counts[name] = count + 1
    return unique_names

df['sample_name_unique'] = make_unique_names(df['sample_name'])

# 3. Expandir casos com "and" e "/"
def expand_multiple_coordinates(df):
    rows = []
    for _, row in df.iterrows():
        coord_str = str(row['longitude'])
        if 'and' in coord_str and '/' in coord_str:
            pairs = [pair.strip() for pair in coord_str.split('and')]
            for i, pair in enumerate(pairs):
                try:
                    easting_raw, northing_raw = pair.split('/')
                    new_row = row.copy()
                    new_row['lon_raw'] = easting_raw.strip()
                    new_row['lat_raw'] = northing_raw.strip()
                    new_row['sample_name_unique'] += f"_{chr(97 + i)}"
                    rows.append(new_row)
                except:
                    continue
        else:
            new_row = row.copy()
            new_row['lat_raw'] = str(row['latitude'])
            new_row['lon_raw'] = str(row['longitude'])
            rows.append(new_row)
    return pd.DataFrame(rows)

df_expanded = expand_multiple_coordinates(df)


###2. Padronização das Coords
Identifica as variações de tipos de coordeandas, padroniza como decimal e corrige erros para manter os dados espacializados na região sul do brasil.

In [5]:
# 4. Conversão UTM para DD
def try_utm_to_dd(northing, easting, zone=22, hemisphere='south'):
    try:
        proj = Proj(proj='utm', zone=zone, south=(hemisphere == 'south'), ellps='WGS84')
        lon, lat = proj(float(easting), float(northing), inverse=True)
        return lat, lon
    except:
        return None, None

# 5. Detecta se valor é UTM
def is_probably_utm(value):
    try:
        num = float(str(value).replace(',', '.').replace(' ', ''))
        return 1000000 < num < 10000000
    except:
        return False

# 6. Conversão genérica para decimal
def convert_to_decimal(coord):
    try:
        coord = str(coord).replace('?', '0').replace(',', '.').strip()

        # 6.1 Hemisfério com símbolo, espaço ou não
        match = re.match(r'(\d+(?:\.\d+)?)\s*°?\s*([NSEW])', coord, re.IGNORECASE)
        if match:
            value = float(match.group(1))
            hemisphere = match.group(2).upper()
            if hemisphere in ['S', 'W']:
                value = -value
            return value

        # 6.2 Decimal direto
        if re.match(r'^-?\d+(\.\d+)?$', coord):
            return float(coord)

        # 6.3 DMS
        dms = re.findall(r'-?\d+', coord)
        if len(dms) >= 3:
            d, m, s = map(float, dms[:3])
            if not (0 <= m < 60 and 0 <= s < 60):
                return None
            dd = abs(d) + m / 60 + s / 3600
            return -dd if '-' in coord else dd

        # 6.4 DM
        elif len(dms) == 2:
            d, m = map(float, dms)
            if not (0 <= m < 60):
                return None
            dd = abs(d) + m / 60
            return -dd if '-' in coord else dd

        return None
    except:
        return None

# 7. Conversão linha a linha
def smart_coord_conversion(row):
    lat_raw = str(row['lat_raw']).replace(' ', '')
    lon_raw = str(row['lon_raw']).replace(' ', '')

    lat_dec = convert_to_decimal(lat_raw)
    lon_dec = convert_to_decimal(lon_raw)

    # Correção de sinal faltando para valores que parecem do Brasil
    if lat_dec and lat_dec > 0 and lat_dec < 34:
        lat_dec = -lat_dec
    if lon_dec and lon_dec > 0 and lon_dec < 60:
        lon_dec = -lon_dec

    # Se ainda falhar, tenta UTM
    if (lat_dec is None or lon_dec is None) and is_probably_utm(lat_raw) and is_probably_utm(lon_raw):
        try:
            northing = float(re.sub(r'[^\d.]', '', lat_raw))
            easting = float(re.sub(r'[^\d.]', '', lon_raw))
            lat_dec, lon_dec = try_utm_to_dd(northing, easting)
        except:
            lat_dec, lon_dec = None, None

    return pd.Series({'lat_dd': lat_dec, 'lon_dd': lon_dec})

# 8. Aplicar conversão
df_coords = df_expanded.apply(smart_coord_conversion, axis=1)
df_final = pd.concat([df_expanded, df_coords], axis=1)

# 9. Remover coordenadas inválidas
df_final = df_final.dropna(subset=['lat_dd', 'lon_dd'])

# 9.1 Filtrar apenas para a região sul do Brasil
df_final = df_final[
    df_final['lat_dd'].between(-34, -23) &
    df_final['lon_dd'].between(-57, -47)
]
# 9.2 Garantir que só amostras com lat/long válidas entrem no GeoDataFrame
df_final = df_final[df_final['lat_dd'].notnull() & df_final['lon_dd'].notnull()]


###3. Criação do GeoDataFrame
Gera a coluna geometry necessária para espacialização e salva os dados em formato shapefile.

In [6]:
# 10. Criar GeoDataFrame
geometry = [Point(xy) for xy in zip(df_final['lon_dd'], df_final['lat_dd'])]
gdf = gpd.GeoDataFrame(df_final, geometry=geometry, crs='EPSG:4326')

# 11. Exportar para shapefile
gdf.to_file("dados_convertidos.shp", driver="ESRI Shapefile", encoding="utf-8")

# 12. Visualizar as primeiras linhas
print(gdf[['sample_name_unique', 'lat_dd', 'lon_dd', 'geometry']].head())


/tmp/ipython-input-6-560822401.py:6: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file("dados_convertidos.shp", driver="ESRI Shapefile", encoding="utf-8")
/usr/local/lib/python3.11/dist-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'sample_name' to 'sample_nam'
  ogr_write(
/usr/local/lib/python3.11/dist-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'suite_by_autor' to 'suite_by_a'
  ogr_write(
/usr/local/lib/python3.11/dist-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'classification_peate92' to 'classifica'
  ogr_write(
/usr/local/lib/python3.11/dist-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'classification_peate97' to 'classifi_1'
  ogr_write(
/usr/local/lib/python3.11/dist-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'localization' to 'localizati'
  ogr_wr

    sample_name_unique  lat_dd  lon_dd              geometry
433             GB17B   -28.36   -49.6  POINT (-49.6 -28.36)
434              GB16   -28.36   -49.6  POINT (-49.6 -28.36)
435             GB25A   -28.36   -49.6  POINT (-49.6 -28.36)
436               GB8   -28.36   -49.6  POINT (-49.6 -28.36)
437              GB15   -28.36   -49.6  POINT (-49.6 -28.36)
